In [1]:
# 导入读取数据所需要的库
import numpy as np
import pandas as pd
from pandas import read_excel

df = read_excel(r'data\SEV_wordlist.xlsx')

In [2]:
import gensim
from gensim import models

In [3]:
# 获取Excel文件中所有表的名称
excel_file = pd.ExcelFile(r'data\SEV_wordlist.xlsx')
print(excel_file.sheet_names)  # 显示所有表的名称

['App_words', 'SES_words', 'Soc_words', 'Com_words', 'Mor_words']


In [4]:
# 预训练的词向量语料库文件路径
tecent_embedding_path = (r'D:\embedding_zh\tencent-ailab-embedding-zh-d200-v0.2.0-s\tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')

# 加载词向量文件
tencent_embedding = models.KeyedVectors.load_word2vec_format(tecent_embedding_path, binary=False)

In [6]:
# Soc扩展词表

df_Soc_words = pd.read_excel(r'data\SEV_wordlist.xlsx', sheet_name=2)  # 表的索引从0开始，1表示第二张表

# 查看数据的形状
df_Soc_words.shape

# 预览数据
df_Soc_words.head()

,Word,Dimension,Soc_Dim_mean,Soc_Dim_sd,App_Dim_mean,App_Dim_sd,SES_Dim_mean,SES_Dim_sd,Com_Dim_mean,Com_Dim_sd,...,Val_sd,Valence,N_raters,N_App,N_SES,N_Soc,N_Com,N_Mor,N_Non,N_Amb
0,交际花,Sociability,8.019,1.829,4.220096,2.862116,4.928230,2.740301,5.239234,2.922096,...,1.816,Positive,209,8,5,108,6,1,9,72
1,合群,Sociability,7.493,2.215,2.251185,1.926862,3.322275,2.520474,4.421801,2.838057,...,1.370,Positive,211,3,4,145,16,2,14,27
2,能说会道,Sociability,7.456,2.111,2.641860,2.165385,3.279070,2.425773,6.660465,2.579768,...,1.530,Positive,215,4,0,81,41,5,9,75
3,左右逢源,Sociability,7.223,2.405,2.125581,1.758304,4.469767,2.733962,5.558140,2.952128,...,1.903,Positive,215,0,10,90,19,7,19,70
4,外向,Sociability,7.223,2.343,3.497674,2.624248,2.604651,2.132739,3.702326,2.585861,...,1.460,Positive,215,10,3,136,7,6,21,32


In [7]:
# 求积极词汇的词表
df_Soc_positive = df_Soc_words.loc[df_Soc_words['Valence']=='Positive', 'Word']
df_Soc_positive.head()

0     交际花
1      合群
2    能说会道
3    左右逢源
4      外向
Name: Word, dtype: object

In [8]:
# 格式转化为列表
Soc_positive_word_list = df_Soc_positive.tolist()
Soc_positive_word_list

['交际花',
 '合群',
 '能说会道',
 '左右逢源',
 '外向',
 '吃得开',
 '健谈',
 '热情',
 '平易近人',
 '察言观色',
 '好客',
 '随和',
 '主动',
 '好说话',
 '活跃',
 '幽默',
 '眼力见儿',
 '世故',
 '伶牙俐齿',
 '直爽',
 '风趣',
 '爽快',
 '豪爽',
 '活泼',
 '开放',
 '亲切',
 '爽朗',
 '够意思',
 '投其所好',
 '有趣',
 '心直口快',
 '率直',
 '合作',
 '爽直',
 '从容',
 '热诚',
 '直性子',
 '大大咧咧',
 '和气',
 '直率',
 '识相',
 '识趣',
 '直肠子',
 '客气',
 '鬼灵精',
 '高谈阔论',
 '直心眼儿',
 '贴心',
 '开朗',
 '不拘小节',
 '走心',
 '率然',
 '亲近',
 '害羞',
 '直性',
 '外圆内方',
 '诙谐',
 '暖男',
 '体贴',
 '豪放',
 '知趣',
 '率性',
 '和顺',
 '通透']

In [9]:
# 初始化有效种子词列表和相似词列表
Soc_positive_word_list_effective = []
Soc_similar_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in Soc_positive_word_list:
    if word in tencent_embedding:
        Soc_positive_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        Soc_similar_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_word, similarity in Soc_similar_words:
            Soc_similar_words_list.append(similar_word)
    else:
        print(f"Skipping {word}...")

Skipping 直心眼儿...
Skipping 率然...


In [11]:
Soc_unique_similar_words_list = list(set(Soc_similar_words_list))

In [12]:
df_unique_Soc_similar_words = pd.DataFrame(Soc_unique_similar_words_list, columns=['Word'])
print(df_unique_Soc_similar_words)

     Word
0    随性不羁
1    不会主动
2    咋咋呼呼
3    性格直爽
4    嘻嘻哈哈
..    ...
501   不识相
502  舌灿莲花
503  十分活跃
504   好男友
505  豪迈不羁

[506 rows x 1 columns]


In [13]:
df_unique_Soc_similar_words.to_excel(r'result\Soc_unique_positive_similar_words.xlsx', index=False)

In [14]:
# 消极词汇
df_Soc_negative = df_Soc_words.loc[df_Soc_words['Valence']=='Negative', 'Word']
df_Soc_negative.head()

64      圆滑
65      内向
66    笨嘴拙舌
67      油滑
68    老于世故
Name: Word, dtype: object

In [15]:
# 格式转化为列表
Soc_negative_word_list = df_Soc_negative.tolist()

# 初始化有效种子词列表和相似词列表
Soc_negative_word_list_effective = []
Soc_similar_negative_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in Soc_negative_word_list:
    if word in tencent_embedding:
        Soc_negative_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        Soc_similar_negative_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_negative_word, similarity in Soc_similar_negative_words:
            Soc_similar_negative_words_list.append(similar_negative_word)
    else:
        print(f"Skipping {word}...")

Skipping 笨口拙舌...
Skipping 贫嘴薄舌...
Skipping 狐朋狗党...


In [16]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
Soc_unique_similar_negative_words_list = list(set(Soc_similar_negative_words_list))

# 将去重后的结果转换为 DataFrame
df_Soc_unique_similar_negative_words = pd.DataFrame(Soc_unique_similar_negative_words_list, columns=['Word'])
print(df_Soc_unique_similar_negative_words)

     Word
0    畏畏缩缩
1    讨好巴结
2      畏怯
3     讨人厌
4    略显孤单
..    ...
344    逢迎
345  舌灿莲花
346    嘴笨
347   惹人烦
348    呆板

[349 rows x 1 columns]


In [17]:
# 保存去重后的相似词表到excel文件
df_Soc_unique_similar_negative_words.to_excel(r'result\Soc_unique_negative_similar_words.xlsx', index=False)